In [ ]:
import festim as F
import fenics as f
import h_transport_materials as htm
import numpy as np
from scipy.optimize import curve_fit
import itertools

In [ ]:
# Nickel material properties and dimensions
nickel_diffusivity = htm.diffusivities.filter(material=htm.NICKEL).mean()
nickel_solubility = htm.solubilities.filter(material=htm.NICKEL).mean()
nickel_permeability = htm.permeabilities.filter(material=htm.NICKEL).mean()
nickel_length = 2e-3 # in m

nickel = F.Material(
    id=1,
    D_0 = nickel_diffusivity.pre_exp.magnitude,
    E_D = nickel_diffusivity.act_energy.magnitude,
    S_0 = nickel_solubility.pre_exp.magnitude,
    E_S = nickel_solubility.act_energy.magnitude,
)



In [ ]:
# FLiBe material properties and dimensions
flibe_diffusivity = htm.diffusivities.filter(material=htm.FLIBE, author='calderoni')[0]
flibe_solubility = htm.solubilities.filter(material=htm.FLIBE, author='calderoni')[0]
flibe_permeability = htm.permeabilities.filter(material=htm.FLIBE).mean()
flibe_length = 8e-3

flibe = F.Material(
    id=1,
    D_0=flibe_diffusivity.pre_exp.magnitude,
    E_D=flibe_diffusivity.act_energy.magnitude,
    S_0 = flibe_solubility.pre_exp.magnitude,
    E_S = flibe_solubility.act_energy.magnitude,
)

In [ ]:
# Building and running the nickel model
nickel_model = F.Simulation()

# 2mm thick mesh
nickel_model.mesh = F.MeshFromVertices(
    vertices=np.linspace(0, nickel_length, num=1001)
)
nickel_model.materials = nickel
nickel_model.T = F.Temperature(value=973)

P_up = 101325  # Pa

nickel_model.boundary_conditions = [
    F.SievertsBC(surfaces=1, S_0=nickel.S_0, E_S=nickel.E_S, pressure=P_up),
    F.DirichletBC(surfaces=2, value=0, field=0)
]

nickel_model.settings = F.Settings(
    absolute_tolerance=1e10,
    relative_tolerance=1e-10,
    final_time=1 * 3600 # s
    )

nickel_model.dt = F.Stepsize(initial_value=10, stepsize_change_ratio=1.1)

derived_quantities = F.DerivedQuantities([F.HydrogenFlux(surface=2)])

folder = "1D_model"
nickel_model.exports = [
        F.XDMFExport(field="solute", filename=f"{folder}/nickel_concentration.xdmf", checkpoint=False),
        derived_quantities,
    ]

In [ ]:
# Running the model

nickel_model.initialise()
nickel_model.run()

In [ ]:
# Defining the downstream flux for salt and metal

def downstream_flux_salt(t, P_up, L, permeability, D):
    """calculates the downstream H flux at a given time t

    Args:
        t (float, np.array): the time
        P_up (float): upstream partial pressure of H
        permeability (float): salt permeability
        L (float): salt thickness
        D (float): diffusivity of H in the salt

    Returns:
        float, np.array: the downstream flux of H
    """
    n_array = np.arange(1, 10000)[:, np.newaxis]
    summation = np.sum((-1)**n_array * np.exp(-(np.pi * n_array)**2 * D/L**2 * t), axis=0)
    return P_up * permeability / L * (2*summation + 1)

def downstream_flux_metal(t, P_up, permeability, L, D):
    """calculates the downstream H flux at a given time t

    Args:
        t (float, np.array): the time
        P_up (float): upstream partial pressure of H
        permeability (float): material permeability
        L (float): material thickness
        D (float): diffusivity of H in the material

    Returns:
        float, np.array: the downstream flux of H
    """
    n_array = np.arange(1, 10000)[:, np.newaxis]
    summation = np.sum((-1)**n_array * np.exp(-(np.pi * n_array)**2 * D/L**2 * t), axis=0)
    return P_up**0.5 * permeability / L * (2*summation + 1)

In [ ]:
# Plotting the model and the analytical solution
times = derived_quantities.t
computed_flux = derived_quantities.filter(surfaces=2).data

import matplotlib.pyplot as plt
T = 973.15 * htm.ureg.K
plt.scatter(times, np.abs(computed_flux), alpha=0.2, label="Computed")
plt.plot(times, downstream_flux_metal(times * htm.ureg.sec, P_up * htm.ureg.Pa,
                                permeability=nickel_diffusivity.value(T)*nickel_solubility.value(T), 
                                L=nickel_length*htm.ureg.m, D=nickel_diffusivity.value(T)), color="tab:orange", label="Analytical")
plt.ylim(bottom=0)
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Downstream flux (H/m2/s)")
plt.show()

In [ ]:
# Building, running, and plotting the flibe model
# Building and running the nickel model
flibe_model = F.Simulation()

flibe_length = 8.1e-3
# 20mm thick mesh
flibe_model.mesh = F.MeshFromVertices(
    vertices=np.linspace(0, flibe_length, num=1001)
)
flibe_model.materials = flibe
flibe_model.T = F.Temperature(value=973)

P_up = 100  # Pa

flibe_model.boundary_conditions = [
    F.HenrysBC(surfaces=1, H_0=flibe.S_0, E_H=flibe.E_S, pressure=P_up),
    F.DirichletBC(surfaces=2, value=0, field=0)
]

flibe_model.settings = F.Settings(
    absolute_tolerance=1e10,
    relative_tolerance=1e-10,
    final_time=6 * 3600 # s
    )

flibe_model.dt = F.Stepsize(initial_value=10, stepsize_change_ratio=1.1)
flibe_derived_quantities = F.DerivedQuantities([F.HydrogenFlux(surface=2)])
flibe_model.exports = [
        F.XDMFExport(field="solute", filename=f"{folder}/flibe_concentration.xdmf", checkpoint=False),
        flibe_derived_quantities,
    ]
flibe_model.initialise()
flibe_model.run()

times = flibe_derived_quantities.t
flibe_computed_flux = flibe_derived_quantities.filter(surfaces=2).data

# Checking curve fit works for regular data
guess = (flibe_diffusivity.value(T).magnitude*flibe_solubility.value(T).magnitude, flibe_diffusivity.value(T).magnitude)
props, cov = curve_fit(lambda t, permeability, D: downstream_flux_salt(t, P_up, flibe_length, permeability, D), times, np.abs(flibe_computed_flux), guess)
print('props', props)
print('real', guess)


T = flibe_model.T.value * htm.ureg.K
plt.scatter(times, np.abs(flibe_computed_flux), alpha=0.3, label="computed")
plt.plot(times, downstream_flux_salt(times * htm.ureg.sec, P_up * htm.ureg.Pa,
                                permeability=flibe_diffusivity.value(T)*flibe_solubility.value(T), 
                                L=flibe_length*htm.ureg.m, D=flibe_diffusivity.value(T)), color="tab:orange", label="analytical")
plt.ylim(bottom=0)
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Downstream flux (H/m2/s)")
plt.title("1D salt only flux as a function of time")
####plt.savefig('/Users/jaron/Downloads/1d_one_mat.svg')
plt.show()

In [ ]:
# Adding in nickel as a permeation barrier, method from task04
# Changing material IDs and assigning borders

# Adjusting parameters for the Calderoni experiment

flibe_length = 8.1e-3
P_up = 100
nickel_length = 2e-3

nickel.borders = [0, nickel_length]

flibe.borders = [nickel_length, nickel_length + flibe_length]
flibe.id = 2
flibe.solubility_law = "henry"

model_barrier = F.Simulation()
model_barrier.materials = [nickel, flibe]

vertices_bottom = np.linspace(0, nickel_length, num = 500)
vertices_top = np.linspace(nickel_length, nickel_length + flibe_length, num = 500)
vertices = np.concatenate([vertices_bottom, vertices_top])

model_barrier.mesh = F.MeshFromVertices(vertices)

model_barrier.T = F.Temperature(973)

bottom_bc = F.SievertsBC(
    surfaces=1,
    S_0=nickel.S_0,
    E_S=nickel.E_S,
    pressure = P_up
)

# Assuming zero concentration on the top surface
top_bc = F.DirichletBC(
    field="solute",
    surfaces=2,
    value=0
)

model_barrier.boundary_conditions = [bottom_bc, top_bc]

derived_quantities_with_barrier = F.DerivedQuantities([F.HydrogenFlux(surface=2)])
txt_export = F.TXTExport(
    field='solute',
    filename=f"{folder}/barrier_concentration.txt", 
    times = [0.1*3600,0.2*3600,1*3600, 6*3600])
model_barrier.exports = [
        F.XDMFExport(field="solute", filename=f"{folder}/barrier_concentration.xdmf", checkpoint=False),
        derived_quantities_with_barrier,
        txt_export
    ]

model_barrier.settings = F.Settings(
    absolute_tolerance=1e8,
    relative_tolerance=1e-10,
    final_time=8*3600, # 6 hours
    chemical_pot=True,
)

model_barrier.dt = F.Stepsize(
    initial_value=10,
    stepsize_change_ratio=1.1,
)

In [ ]:
# Running the model
model_barrier.initialise()
model_barrier.run()

In [ ]:
# Plotting concentration as a function of depth at various times
# Method derived from task04

import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10,5))
data = np.genfromtxt(folder + f"/barrier_concentration.txt", skip_header=1, delimiter=",")
data = data[data[:, 0].argsort()]  # make sure data is sorted

xlim_left = (0, nickel_length * 1.2)
xlim_right = (0, (nickel_length + flibe_length) * 1.2)

for ax, xlim in zip(
    axs,
    [xlim_left, xlim_right],
):
    plt.sca(ax)
    for i, time in enumerate(txt_export.times):
        plt.plot(data[:, 0], data[:, i + 1], label=f"{time:.0f} s")

    plt.xlabel("Depth (m)")
    plt.xlim(*xlim)

axs[0].set_yscale("log")
axs[0].set_ylim(bottom=1e20)
axs[0].set_ylabel("Mobile H concentration (H/m$^3$)")
axs[0].set_title("zoom left")
axs[1].set_title("total")
axs[0].legend(reverse=True)
#plt.savefig("/Users/jaron/Downloads/concentration_depth.svg")
plt.show()

In [ ]:
# Plotting the model and the analytical solution to the flibe
times = derived_quantities_with_barrier.t
barrier_computed_flux = derived_quantities_with_barrier.filter(surfaces=2).data
N_A = 6.022e-23

T = 973.15 * htm.ureg.K
plt.figure()
plt.scatter(times, np.abs(barrier_computed_flux)*N_A, alpha=0.5, label="computed", marker = ".")

plt.plot(times, downstream_flux_salt(times * htm.ureg.sec, P_up * htm.ureg.Pa,
                                permeability=flibe_diffusivity.value(T)*flibe_solubility.value(T), 
                                L=flibe_length*htm.ureg.m, D=flibe_diffusivity.value(T))*N_A, color="tab:red", label="no barrier")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Downstream flux (mol H/m2/s)")
plt.title("Molar flux with nickel as a permeation barrier")
plt.show()

### Now to figure out the error from the boundary
The plan:
- Figure out a curve fit and have it extract the permeability and the diffusivity and thus the solubility
- calculate the error between the analytical props and the actual props (comparing it down downstream flux of flibe only)
    - bc if negligible then we dont care about nickel and only care about flibe
- run a bunch of simulations with various params
- plot the error as a function of the params
- 3d plot/gradient of error as the z and pressure/thickness as x/y
    - probably have to 3d interpolate values or something to get a smooth gradient

In [ ]:
from scipy.optimize import curve_fit
#print(times)
#print(np.abs(barrier_computed_flux))

# Proof of concept
guess = (3157333279361, 5.090185116256628e-09)
props, cov = curve_fit(lambda t, permeability, D: downstream_flux_salt(t, P_up, flibe_length, permeability, D), times, np.abs(barrier_computed_flux), guess)
plt.scatter(times, np.abs(barrier_computed_flux), alpha=0.5, label="Computed two material flux", marker = ".")
plt.plot(times, downstream_flux_salt(times, P_up, flibe_length, *props), label="Curve fit")
plt.plot(times, downstream_flux_salt(times * htm.ureg.sec, P_up * htm.ureg.Pa, L=flibe_length*htm.ureg.m, permeability=flibe_diffusivity.value(T)*flibe_solubility.value(T), D=flibe_diffusivity.value(T)), color="tab:red", label="One material (salt only)")
plt.legend()
perm=flibe_diffusivity.value(T).magnitude*flibe_solubility.value(T).magnitude
sol = props[0]/props[1]

print("diff calderoni", flibe_diffusivity.value(T))
print("sol", flibe_solubility.value(T))
print("diff calc", props[1])
print("perm calc", props[0])
print("perm calderoni", perm)
print("sol calc", sol)
print("sol calderoni", flibe_solubility.value(T).magnitude)
diff_error = np.abs(flibe_diffusivity.value(T).magnitude - props[1])/flibe_diffusivity.value(T).magnitude*100
print("diff error", diff_error)
perm_error = np.abs(perm - props[0])/perm*100
print("perm error", perm_error)
sol_error = np.abs(flibe_solubility.value(T).magnitude - sol)/flibe_solubility.value(T).magnitude*100
print("sol error", sol_error)

plt.title("Comparison of one material and two material fluxes")
plt.savefig('/Users/jaron/Downloads/1d_two_mat.svg')
print(*props)

Now wrapping the above cells in functions for improved functionality

### Make model function

In [ ]:
def make_model(nickel_length, flibe_length, temp, P_up, folder = '1D_model'):
    '''
    Function to make 1D festim model
    nickel_length: length of nickel in [meters]
    flibe_length: length of flibe [meters]
    temp: temp of experiment [Kelvin]
    P_up: upstream hydrogen partial pressure [Pa]
    
    Runs the model and returns the derived_quantities object of the model
    '''
    nickel.borders = [0, nickel_length]
    flibe.borders = [nickel_length, nickel_length + flibe_length]
    model = F.Simulation()
    model.materials = [nickel, flibe]
    vertices_bottom = np.linspace(0, nickel_length, num = 500)
    vertices_top = np.linspace(nickel_length, nickel_length + flibe_length, num = 500)
    vertices = np.concatenate([vertices_bottom, vertices_top])
    model.mesh = F.MeshFromVertices(vertices)
    model.T = F.Temperature(temp)

    bottom_bc = F.SievertsBC(
        surfaces=1,
        S_0=nickel.S_0,
        E_S=nickel.E_S,
        pressure = P_up
    )

    # Assuming zero concentration on the top surface
    top_bc = F.DirichletBC(
        field="solute",
        surfaces=2,
        value=0
    )

    model.boundary_conditions = [bottom_bc, top_bc]

    derived_quantities = F.DerivedQuantities([F.HydrogenFlux(surface=2)])
    txt_export = F.TXTExport(
        field='solute',
        filename=f"{folder}/barrier_concentration.txt", 
        times = [0.1*3600,0.2*3600,1*3600, 6*3600])
    
    model.exports = [
        F.XDMFExport(field="solute", filename=f"{folder}/barrier_concentration.xdmf", checkpoint=False),
        derived_quantities,
        txt_export
    ]
    model.settings = F.Settings(
        absolute_tolerance=1e8,
        relative_tolerance=1e-10,
        final_time=20*3600, # 8 hours
        chemical_pot=True,
    )
    
    model.dt = F.Stepsize(
        initial_value=10,
        stepsize_change_ratio=1.1,
    )

    model.initialise()
    model.run() 

    return derived_quantities

In [ ]:
model = make_model(nickel_length, flibe_length, T.magnitude, P_up)


### Plot model function

In [ ]:
# Function for plotting the values

def plot_concentration(model, nickel_length, flibe_length, temp, P_up, depth=True, time = True):
    '''
    Function for plotting concentration as a function of depth and flux at the top
    surface as a function of time
    model: a model created from the make_model function
    nickel_length: length of nickel [meters]
    flibe_length: length of flibe [meters]
    temp: temperature of experiment [Kelvin]
    P_up: upstream hydrogen partial pressure [Pa]
    depth: give depth graph
    time: give time graph

    '''
    if depth:
        fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(10,5))
        data = np.genfromtxt(folder + f"/barrier_concentration.txt", skip_header=1, delimiter=",")
        data = data[data[:, 0].argsort()]  # make sure data is sorted

        xlim_left = (0, nickel_length * 1.2)
        xlim_right = (0, (nickel_length + flibe_length) * 1.2)

        for ax, xlim in zip(
            axs,
            [xlim_left, xlim_right],
        ):
            plt.sca(ax)
            for i, time in enumerate(txt_export.times):
                plt.plot(data[:, 0], data[:, i + 1], label=f"{time:.0f} s")

            plt.xlabel("Depth (m)")
            plt.xlim(*xlim)

        axs[0].set_yscale("log")
        axs[0].set_ylim(bottom=1e20)
        axs[0].set_ylabel("Mobile H concentration (H/m$^3$)")
        axs[0].set_title("zoom left")
        axs[1].set_title("total")
        axs[0].legend(reverse=True)
        plt.show()

    if time:
        times = model.t
        barrier_computed_flux = model.filter(surfaces=2).data

        T = temp * htm.ureg.K
        plt.figure()
        plt.scatter(times, np.abs(barrier_computed_flux), alpha=0.5, label="computed", marker = ".")

        plt.plot(times, downstream_flux_salt(times * htm.ureg.sec, P_up * htm.ureg.Pa,
                                        permeability=flibe_diffusivity.value(T)*flibe_solubility.value(T), 
                                        L=flibe_length*htm.ureg.m, D=flibe_diffusivity.value(T)), color="tab:red", label="no barrier")
        plt.legend()
        plt.xlabel("Time (s)")
        plt.ylabel("Downstream flux (mol H/m2/s)")
        plt.title("Molar flux with nickel as a permeation barrier")
        plt.show()
    return

### Error calculation

In [ ]:
# Function for fitting the curve and giving the errors of the expected vs barrier

def prop_errors(model, flibe_length, temp, P_up, plot = False, label = None):
    '''
    Function for quantifying the error between the observed properties and the properties
    that are calculated with the barrier

    Returns a dictionary of "diffusivity error", "solubility error", and "permeability error"

    model: a model created with the make_model function
    flibe_length: the length of flibe [meters]
    temp: the temperature of the experiment [Kelvin]
    plot: plot the data and the fitted curve
    label: a string that adds a label to the legend of the plot
    '''
    # Providing a guess for the curve_fit function, which is the (measured perm, measured diff)
    guess = (flibe_solubility.value(temp).magnitude*flibe_diffusivity.value(temp).magnitude, flibe_diffusivity.value(temp).magnitude)

    computed_flux = model.filter(surfaces=2).data
    times = model.t

    # Using a lambda function so that the length and the pressure are not being fit
    props, cov = curve_fit(lambda t, permeability, D: downstream_flux_salt(t, P_up, flibe_length, permeability, D), times, np.abs(computed_flux), guess)    

    # The calculated solubility and permeability
    perm = flibe_diffusivity.value(temp).magnitude * flibe_solubility.value(temp).magnitude
    sol = props[0]/props[1]

    # The error calculations
    diff_error = np.abs(flibe_diffusivity.value(temp).magnitude - props[1])/flibe_diffusivity.value(temp).magnitude*100
    sol_error = np.abs(flibe_solubility.value(temp).magnitude - sol)/flibe_solubility.value(temp).magnitude*100
    perm_error = np.abs(perm - props[0])/perm*100


    # Plotting functionality
    if plot:
        # Having each simulation have the same color on the plot
        marker = itertools.cycle(('o', 'v', '^', '<', '>', 's', '8', 'p'))
        line=plt.plot(times, downstream_flux_salt(times, P_up, flibe_length, *props), linestyle='', markeredgecolor='none', marker=next(marker))
        linecolor = line[0].get_color()

        # Constructing the plots
        plt.scatter(times, np.abs(computed_flux), alpha=0.3, label=f"Computed flux ({label})", marker = ".", color=linecolor)
        plt.plot(times, downstream_flux_salt(times, P_up, flibe_length, *props), label=f"Curve fit ({label})", linestyle = "dashed", color=linecolor)
        plt.plot(times, downstream_flux_salt(times * htm.ureg.sec, P_up * htm.ureg.Pa,permeability=flibe_diffusivity.value(temp)*flibe_solubility.value(temp), L=flibe_length*htm.ureg.m, D=flibe_diffusivity.value(temp)), label=f"No barrier ({label})", color=linecolor)
        plt.legend()
    return {"diffusivity error": diff_error, "solubility error": sol_error, "permeability error": perm_error }

In [ ]:
# Setting up a for loop to sweep through various temperatures and plot the fluxes

nickel_length = 2e-3
flibe_length = 8.1e-3
Pressures = np.linspace(100, 1000, num = 3)
temp = 700+273
lengths = np.linspace(2e-3, 20e-3, num = 3)

'''
for flibe_length in lengths:
    model = make_model(nickel_length, flibe_length, temp, P_up)
    #plot_concentration(model, nickel_length, flibe_length, temp, P_up, depth=False)
    #print("the errors are", prop_errors(model, flibe_length, temp, P_up, label = str(flibe_length) + "m"))
    errs.append(prop_errors(model, flibe_length, temp, P_up)['diffusivity error'])

plt.plot(lengths, errs)
plt.hlines(10, lengths[0], lengths[-1])
plt.show()
'''
"""
errs=[]
for P_up in Pressures:
    model = make_model(nickel_length, flibe_length, temp, P_up)
    errs.append(prop_errors(model, flibe_length, temp, P_up)['diffusivity error'])
"""

overall = []

for P_up in Pressures:
    errs = []
    for flibe_length in lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['diffusivity error'])
    overall.append(errs)






In [ ]:
# Attempting to make a contour

XX, YY = np.meshgrid(lengths, Pressures)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Flibe length (m)")
plt.ylabel("Pressure (Pa)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Diffusivity error by varying pressure and flibe length at given temp")
##plt.savefig('/Users/jaron/Downloads/pressure_flibe_diff.png', dpi = 400)
plt.show()

In [ ]:
# Simulations comparing temperature and flibe length
lengths = np.linspace(2e-3, 20e-3, num = 3)
temps = np.linspace(500+273, 700+273, num = 3)
nickel_length = 2e-3
P_up = 300

overall = []

for temp in temps:
    errs = []
    for flibe_length in lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['diffusivity error'])
    overall.append(errs)

In [ ]:
XX, YY = np.meshgrid(lengths, temps)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Flibe length (m)")
plt.ylabel("Temperature (K)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Diffusivity error by varying temperature and flibe length at given pressure")
##plt.savefig('/Users/jaron/Downloads/temperature_flibe_diff.png', dpi = 400)
plt.show()

In [ ]:
# Simulations comparing pressure and nickel length
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)
Pressures = np.linspace(100, 1000, num = 3)
flibe_length = 8e-3
temp = 700+273

overall = []

for P_up in Pressures:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['diffusivity error'])
    overall.append(errs)

In [ ]:
XX, YY = np.meshgrid(nickel_lengths, Pressures)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 8, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Pressure (Pa)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Diffusivity error by varying pressure and nickel length at given temp")
#plt.savefig('/Users/jaron/Downloads/pressure_nickel_diff.png', dpi = 400)
plt.show()

In [ ]:
# Simulations comparing temperature and nickel length
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)
flibe_length = 8e-3
temps = np.linspace(500+273, 700+237, num = 3)
P_up = 300

overall = []

for temp in temps:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['diffusivity error'])
    overall.append(errs)

In [ ]:
XX, YY = np.meshgrid(nickel_lengths, temps)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Temperature (K)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Diffusivity error by varying temperature and nickel length at given pressure")
#plt.savefig('/Users/jaron/Downloads/temperature_nickel_diff.png', dpi = 400)
plt.show()

In [ ]:
# Varying nickel length and flibe length
flibe_lengths = np.linspace(2e-3, 20e-3, num = 3)
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)

temps = 700+273
P_up = 300

overall = []

for flibe_length in flibe_lengths:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['diffusivity error'])
    overall.append(errs)

In [ ]:
XX, YY = np.meshgrid(nickel_lengths, flibe_lengths)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Flibe length (m)")
plt.xticks(rotation = 45)
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Diffusivity error by varying flibe length and nickel length at given pressure and temperature")
#plt.savefig('/Users/jaron/Downloads/flibe_nickel_diff.png', dpi = 400)
plt.show()

In [ ]:
# Varying nickel length and flibe length
flibe_length = 8e-3
nickel_length = 2e-3

temps = np.linspace(500+273, 700+273, num = 3)
Pressures = np.linspace(100, 1000, num = 3)

overall = []

for P_up in Pressures:
    errs = []
    for temp in temps:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['diffusivity error'])
    overall.append(errs)

In [ ]:
XX, YY = np.meshgrid(temps, Pressures)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Temperature (K)")
plt.ylabel("Pressure (Pa)")
plt.xticks(rotation = 45)
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Diffusivity error by varying pressure and temperature at given lengths")
#plt.savefig('/Users/jaron/Downloads/pressure_temp_diff.png', dpi = 400)
plt.show()

In [ ]:
# Running all the previous simulations for permeability error

nickel_length = 2e-3
flibe_length = 8.1e-3
Pressures = np.linspace(100, 1000, num = 3)
temp = 700+273
lengths = np.linspace(2e-3, 20e-3, num = 3)
overall = []

for P_up in Pressures:
    errs = []
    for flibe_length in lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['permeability error'])
    overall.append(errs)

XX, YY = np.meshgrid(lengths, Pressures)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Flibe length (m)")
plt.ylabel("Pressure (Pa)")
plt.title("Permeability error by varying pressure and flibe length at given temp")
#plt.savefig('/Users/jaron/Downloads/pressure_flibe_perm.png', dpi = 400)
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.show()

# Simulations comparing temperature and flibe length
lengths = np.linspace(2e-3, 20e-3, num = 3)
temps = np.linspace(500+273, 700+273, num = 3)
nickel_length = 2e-3
P_up = 300

overall = []

for temp in temps:
    errs = []
    for flibe_length in lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['permeability error'])
    overall.append(errs)

XX, YY = np.meshgrid(lengths, temps)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Flibe length (m)")
plt.ylabel("Temperature (K)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Permeability error by varying temperature and flibe length at given pressure")
#plt.savefig('/Users/jaron/Downloads/temp_flibe_perm.png', dpi = 400)
plt.show()

# Simulations comparing pressure and nickel length
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)
Pressures = np.linspace(100, 1000, num = 3)
flibe_length = 8e-3
temp = 700+273

overall = []

for P_up in Pressures:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['permeability error'])
    overall.append(errs)

XX, YY = np.meshgrid(nickel_lengths, Pressures)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Pressure (Pa)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Permeability error by varying pressure and nickel length at given temp")
#plt.savefig('/Users/jaron/Downloads/pressure_nickel_perm', dpi = 400)
plt.show()

# Simulations comparing temperature and nickel length
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)
flibe_length = 8e-3
temps = np.linspace(500+273, 700+237, num = 3)
P_up = 300

overall = []

for temp in temps:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['permeability error'])
    overall.append(errs)

XX, YY = np.meshgrid(nickel_lengths, temps)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Temperature (K)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Permeability error by varying temperature and nickel length at given pressure")
#plt.savefig('/Users/jaron/Downloads/temp_nickel_perm.png', dpi = 400)
plt.show()

# Varying nickel length and flibe length
flibe_lengths = np.linspace(2e-3, 20e-3, num = 3)
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)

temps = 700+273
P_up = 300

overall = []

for flibe_length in flibe_lengths:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['permeability error'])
    overall.append(errs)

XX, YY = np.meshgrid(nickel_lengths, flibe_lengths)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Flibe length (m)")
plt.xticks(rotation = 45)
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Permeability error by varying flibe length and nickel length at given pressure and temperature")
#plt.savefig('/Users/jaron/Downloads/flibe_nickel_perm.png', dpi = 400)
plt.show()

In [ ]:
# Running all the previous simulations for solubility error

nickel_length = 2e-3
flibe_length = 8.1e-3
Pressures = np.linspace(100, 1000, num = 3)
temp = 700+273
lengths = np.linspace(2e-3, 20e-3, num = 3)
overall = []

for P_up in Pressures:
    errs = []
    for flibe_length in lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['solubility error'])
    overall.append(errs)

XX, YY = np.meshgrid(lengths, Pressures)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Flibe length (m)")
plt.ylabel("Pressure (Pa)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Solubility error by varying pressure and flibe length at given temperature")
#plt.savefig('/Users/jaron/Downloads/pressure_flibe_sol.png', dpi = 400)
plt.show()

# Simulations comparing temperature and flibe length
lengths = np.linspace(2e-3, 20e-3, num = 3)
temps = np.linspace(500+273, 700+273, num = 3)
nickel_length = 2e-3
P_up = 300

overall = []

for temp in temps:
    errs = []
    for flibe_length in lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['solubility error'])
    overall.append(errs)

XX, YY = np.meshgrid(lengths, temps)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Flibe length (m)")
plt.ylabel("Temperature (K)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Solubility error by varying temperature and flibe length at given pressure")
#plt.savefig('/Users/jaron/Downloads/temp_flibe_sol.png', dpi = 400)
plt.show()

# Simulations comparing pressure and nickel length
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)
Pressures = np.linspace(100, 1000, num = 3)
flibe_length = 8e-3
temp = 700+273

overall = []

for P_up in Pressures:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['solubility error'])
    overall.append(errs)

XX, YY = np.meshgrid(nickel_lengths, Pressures)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Pressure (Pa)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Solubility error by varying pressure and nickel length at given temp")
#plt.savefig('/Users/jaron/Downloads/pressure_nickel_sol.png', dpi = 400)
plt.show()

# Simulations comparing temperature and nickel length
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)
flibe_length = 8e-3
temps = np.linspace(500+273, 700+237, num = 3)
P_up = 300

overall = []

for temp in temps:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['solubility error'])
    overall.append(errs)

XX, YY = np.meshgrid(nickel_lengths, temps)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Temperature (K)")
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Solubility error by varying temperature and nickel length at given pressure")
#plt.savefig('/Users/jaron/Downloads/temp_nickel_sol.png', dpi = 400)
plt.show()

# Varying nickel length and flibe length
flibe_lengths = np.linspace(2e-3, 20e-3, num = 3)
nickel_lengths = np.linspace(0.5e-3, 5e-3, num = 3)

temps = 700+273
P_up = 300

overall = []

for flibe_length in flibe_lengths:
    errs = []
    for nickel_length in nickel_lengths:
        model = make_model(nickel_length, flibe_length, temp, P_up)
        errs.append(prop_errors(model, flibe_length, temp, P_up)['solubility error'])
    overall.append(errs)

XX, YY = np.meshgrid(nickel_lengths, flibe_lengths)
ZZ = overall

CF = plt.contourf(XX, YY, ZZ, levels = 100)
CS = plt.contour(XX,YY,ZZ, levels = 10, colors = 'white')
plt.clabel(CS, fmt="%.2f")
plt.xlabel("Nickel length (m)")
plt.ylabel("Flibe length (m)")
plt.xticks(rotation = 45)
from matplotlib.cm import ScalarMappable
plt.colorbar(CF, label = 'error (%)')
plt.title("Solubility error by varying flibe length and nickel length at given pressure and temperature")
#plt.savefig('/Users/jaron/Downloads/flibe_nickel_sol.png', dpi = 400)
plt.show()